## Problem Statment

You have been provided a dataset that consists of echo-location clicks of two types of whales, namely, Gervais and Cuviers. In this assignment, your task is to classify the different types whales using Gradient Boosting with the help of the XGBoost library. You are expected to fill in functions that would complete this task. We use XGBoost here instead of GradientBoostedTrees in Spark because XGBoost running on a single machine is much faster than Spark running on 10 machines.

The data files were preprocessed on PySpark (10 nodes) cluster. The code for the same can be found in Data_Processing_Whales.ipynb. The preprocessed data is a numpy array with `4175` rows (for the 10mb file) with following columns (zero-indexed):
* Col 0-9: projections on first 10 eigen vectors
* Col 10: rmse
* Col 11: peak2peak
* Col 12: label (`0 if row.species==u'Gervais' else 1`)

You can also refer to XGBoost_Whales.ipynb under for more details on the XGBoost Analysis before you attempt this assignment.

Both Data_Processing_Whales.ipynb and XGBoost_Whales.ipynb can be found under XGBoost directory that was uploaded in edX as a part of "Notebooks for weeks 7 & 8". 

## XGBoost - Theory

A brief overview of gradient boosting in XGBoost can be found here:

* http://xgboost.readthedocs.io/en/latest/model.html
* http://xgboost.readthedocs.io/en/latest/python/python_intro.html



Use the XGBoost API for training and predicting scores: 

* http://xgboost.readthedocs.io/en/latest/python/python_api.html

#### Main API

* `xgboost.train` is the learning API that trains the Gradient Boosting Model,
   * The main parameters are:
      * **plst** – XGBoost parameter list
      * **dtrain** – Data to be trained
      * **num_round** – Number of iterations of boosting. (default: 100)
      * **evallist** – List of items to be evaluated during training
      * **verbose_eval** - This can be used to control how much information the train function prints. You might want to set to **False** to avoid printing logs.
* `bst.predict` is the API that makes score predictions
   * The main parameters are:
      * **dtest** – Test Data
      * **dtrain** – Data to be trained
      * **ntree_limit** – Limit number of trees in the prediction (Use: ntree_limit=bst.best_ntree_limit)
      * **output_margin** - Whether to output the raw untransformed margin value (Use: output_margin=True)

## Notebook Setup

### Importing Required Libraries

In [1]:
%matplotlib inline
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pickle
import random

### Loading Data

In [2]:
with open('Data/X_train.pkl', 'rb') as f:
    X_train = pickle.load(f)

with open('Data/X_test.pkl', 'rb') as f:
    X_test = pickle.load(f)

with open('Data/y_train.pkl', 'rb') as f:
    y_train = pickle.load(f)

with open('Data/y_test.pkl', 'rb') as f:
    y_test = pickle.load(f)

### Setting Parameters for XG Boost
* Maximum Depth of the Tree = 3 _(maximum depth of each decision trees)_
* Step size shrinkage used in update to prevents overfitting = 0.3 _(how to weigh trees in subsequent iterations)_
* Evaluation Criterion= Maximize Loglikelihood according to the logistic regression _(logitboost)_
* Maximum Number of Iterations = 1000 _(total number trees for boosting)_
* Early Stop if score on Validation does not improve for 5 iterations

[Full description of options](https://xgboost.readthedocs.io/en/latest//parameter.html)

In [3]:
#You can change this cell if you wish to, but you aren't expected to
def xgboost_plst():
    param = {}
    param['max_depth']= 3   # depth of tree
    param['eta'] = 0.3      # shrinkage parameter
    param['silent'] = 1     # not silent
    param['objective'] = 'binary:logistic'
    param['nthread'] = 7 # Number of threads used
    param['eval_metric'] = 'logloss'

    plst = param.items()
    return plst

## Exercises

### Computing the score ranges

The function <font color="blue">calc_stats</font> takes the xgboost margin scores as input and returns two numpy arrays min_scr, max_scr which are calculated as follows:

1. **min_scr**: mean - (3 $\times$ std)
2. **max_scr**: mean + (3 $\times$ std)

Here the input margin scores, represents the processed XGBoost margin scores obtained from the <font color="blue">bootstrap_pred</font> function. Each row represents the various scores for a specific example in an iteration and your <font color="blue">calc_stats</font> function is supposed to compute the **min_scr** and **max_scr** as defined for each example. So in the example below, we take a scenario where we have 3 examples which have 4 values each (From 4 bootstrap iterations).


**<font color="magenta" size=2>Example Input</font>**
``` python
[[-0.22 -0.19 -0.17 -0.13][-0.1 -0.05 0.02 0.10][0.03 0.11 0.12 0.15]]
```

Output: min_scr (numpy array), max_scr (numpy array)

**<font color="blue" size=2>Example Output</font>**
``` python
(array([-0.28 -0.23 -0.03]),
 array([-0.08  0.22  0.24]))
```

**Note**: Ensure you round the values in the numpy arrays to two decimal places

In [4]:
def calc_stats(margin_scores):
    x, y = [],[]
    for each in margin_scores:
        t = np.mean(each)
        s = np.std(each)
        x.append(round(t - 3*s,2))
        y.append(round(t + 3*s,2))
    return (np.array(x), np.array(y))
    #
    # YOUR CODE HERE
    #


In [5]:
margin_score = np.array([[-0.22, -0.19, -0.17, -0.13], [-0.1, -0.05, 0.02, 0.10], [0.03, 0.11, 0.12, 0.15]])
min_score, max_score = calc_stats(margin_score)
assert type(min_score) == np.ndarray, 'Incorrect Return type'
assert type(max_score) == np.ndarray, 'Incorrect Return type'

In [6]:
assert (min_score == np.array([-0.28, -0.23, -0.03])).all(), "Incorrect return value"

In [7]:
assert (max_score == np.array([-0.08,  0.22,  0.24])).all(), "Incorrect return value"

In [8]:
# Hidden Tests Here
#
# AUTOGRADER TEST - DO NOT REMOVE
#


In [9]:
# Hidden Tests Here
#
# AUTOGRADER TEST - DO NOT REMOVE
#


### Calculating predictions

Based on the ranges for each of the examples, i.e, (min_scr, max_scr), we can predict whether it's a Gervais or a Cuvier. Since all our scores will be between -1 and +1, we use 0 as the margin line. All examples which are on the left side of the margin, can be classified as Cuvier's and all which are on the right side can be classified as Gervais. However, since we take margin scores from a set of bootstraps for each example, we use the minimum and maximum score arrays to predict to determine the classification.

The function <font color="blue">predict</font> takes the minimum score array and maximum score array and returns predictions as follows:

1. If respective minimum score and maximum score values are less than 0, predict -1 (**Cuvier's**)
2. If respective minimum score value is less than 0 and maximum score value is greater than 0, predict 0 (**Unsure**)
3. If respective minimum score and maximum score values are greater than 0, predict 1 (**Gervais**)

**<font color="magenta" size=2>Example Input</font>**
``` python
min_scr (numpy array) = [-0.78 -0.68 -0.6 -0.53 -0.47 -0.42 -0.32 -0.21 -0.07 0.22]

max_scr (numpy array) = [-0.49 -0.39 -0.33 -0.25 -0.2 -0.11 -0.04 0.1 0.3 0.51]
```
Output: pred (numpy array of predictions)

**<font color="blue" size=2>Example Output</font>**
``` python
[-1 -1 -1 -1 -1 -1 -1  0  0  1]
```

In [10]:
def predict(min_scr, max_scr):
    x = []
    for i, j in zip(min_scr, max_scr):
        if i<0 and j<0:
            x.append(-1)
        elif i<0 and j>0:
            x.append(0)
        elif i>0 and j>0:
            x.append(1)
    return np.array(x)
                
    #
    # YOUR CODE HERE
    #


In [11]:
max_s = np.array([-0.49, -0.39, -0.33, -0.25, -0.2, -0.11, -0.04, 0.1, 0.3, 0.51])
min_s = np.array([-0.78, -0.68, -0.6, -0.53, -0.47, -0.42, -0.32, -0.21, -0.07, 0.22])
pred = predict(min_s, max_s)
true_pred = np.array([-1, -1, -1, -1, -1, -1, -1, 0, 0, 1])


In [12]:
assert type(pred) == np.ndarray, 'Incorrect return type'

In [13]:
assert (pred == true_pred).all(), 'Incorrect return value'

### Calculating scores

The function <font color="blue">bootstrap_pred</font> takes as input:

1. **Training set**
1. **Test set**
1. **n_bootstrap** Number of bootstrap samples that run XGBoost and trains one part of the sample set.
1. **minR, maxR** two numbers such that $0 < minR < maxR < 1$ that define the fractions of the `n_bootstrap` scores that define the range.
1. **bootstrap_size** - Number of bootstrap samples on which you will run XGBoost.
1. **num_round** - Number of iterations for running xgboost
1. **plst** - Parameter List

The output should be a confidence interval for each example in the test set. Together with a prediction that is `Gervais / Cuviers / Unsure`. The prediction `unsure` is to be output if the confidence interval contains the point 0.
After generating the confidence intervals, the function <font color="blue">predict</font> can be used to make predictions.

**Procedure**

Repeat the given procedure for n_bootstrap number of iterations:

For **n_bootstrap** iterations:
* Sample **boostrap_size** indices from the training set **with replacmennt**
* Create train and test data matrices (dtrain, dtest) using xgb.DMatrix(X_sample, label=y_sample)
* Initialise the evallist parameter [(dtrain, 'train'), (dtest, 'eval')]
* Train the model using the XGBoost train API and make score predictions using bst.predict object returned by XGB train API. Ensure you set **output_margin=True** to get raw untransformed output scores.
* Normalize them by dividing them with the normalizing factor as max(scores) - min(scores) and round these values to a precision of two decimal places

Then: 
* For each individual example, remove scores below the minRth percentile and greater than maxRth percentile (sort for each example if necessary)
* Call the calc_stats function to compute min_scr and max_scr with the filtered margin scores as parameter
* Return the min_scr and max_scr computed by the **calc_stat** function using the margin scores.
      
**Note**: You can experiment by changing **n_bootstraps**, but it takes about 200 iterations to get consistent values.

In [14]:
def bootstrap_pred(X_train, X_test, y_train, y_test, n_bootstrap, minR, maxR, bootstrap_size, \
                   num_round=100, plst=xgboost_plst()):
    lst_train = list(range(len(X_train)))
    out=[[]]*len(X_test)
    pop_lst = []
    for dummy_i in range(n_bootstrap):
        idx = np.random.choice(lst_train,size=bootstrap_size,replace=True)
        sub_x, sub_y = X_train[idx], y_train[idx]
        dtrain = xgb.DMatrix(sub_x, label=sub_y)
        dtest = xgb.DMatrix(X_test, label=y_test)
        evallist = [(dtest, 'eval'), (dtrain, 'train')]
        bst = xgb.train(plst, dtrain, num_round, evallist)
        tmp_out = bst.predict(dtest,output_margin=True)
        tmp_out=np.round(tmp_out / (tmp_out.max() - tmp_out.min()) ,2)
        pop_lst.append(np.copy(tmp_out).tolist())
    x=np.array(pop_lst)
    xx=x.T
    xx.sort(axis=1)
    qlist=xx.tolist()
    qlst2=[]
    for each1 in qlist:
        qlst1=[]
        q01=np.percentile(each1,10)
        q09=np.percentile(each1,90)
        for each2 in each1:
            if each2>=q01 or each2<=q09:
                qlst1.append(each2)
        qlst2.append(qlst1)        
    min_scr,max_scr = calc_stats(qlst2)
    return min_scr,max_scr

In [15]:
def process(X_train, X_test, y_train, y_test, n_bootstrap=100):
    min_scr, max_scr = bootstrap_pred(X_train, X_test, y_train, y_test, n_bootstrap=n_bootstrap, \
                                            minR=0.1, maxR=0.9, bootstrap_size=len(X_train))
    pred = predict(min_scr, max_scr)
    return min_scr, max_scr, pred

#### Tests

How we test the function:
1. We have computed the average mid-point of the range of values and verify that this midpoint is present in the range computed by your function
2. We check that the length of the interval(max_scr-min_scr) is not more than twice the average length of the interval

In [16]:
sample_indices = np.load('Data/vis_indices.npy')
X_test_samp = X_test[sample_indices]
y_test_samp = np.array(y_test[sample_indices], dtype=int)
midpt = np.load('Data/vis_midpt.npy')
avg_length = np.load('Data/vis_avg_length.npy')
min_scr, max_scr, pred = process(X_train, X_test_samp, y_train, y_test_samp)
length = max_scr - min_scr

[0]	eval-logloss:0.626548	train-logloss:0.555661
[1]	eval-logloss:0.599557	train-logloss:0.480443
[2]	eval-logloss:0.609094	train-logloss:0.431249
[3]	eval-logloss:0.617757	train-logloss:0.397679
[4]	eval-logloss:0.599425	train-logloss:0.374235
[5]	eval-logloss:0.651384	train-logloss:0.357401
[6]	eval-logloss:0.645764	train-logloss:0.340977
[7]	eval-logloss:0.636321	train-logloss:0.329919
[8]	eval-logloss:0.594277	train-logloss:0.316774
[9]	eval-logloss:0.614139	train-logloss:0.308078
[10]	eval-logloss:0.631309	train-logloss:0.3011
[11]	eval-logloss:0.643003	train-logloss:0.291981
[12]	eval-logloss:0.654258	train-logloss:0.285383
[13]	eval-logloss:0.642997	train-logloss:0.280592
[14]	eval-logloss:0.674001	train-logloss:0.275024
[15]	eval-logloss:0.655659	train-logloss:0.2691
[16]	eval-logloss:0.645703	train-logloss:0.265141
[17]	eval-logloss:0.659725	train-logloss:0.260367
[18]	eval-logloss:0.693812	train-logloss:0.256668
[19]	eval-logloss:0.680656	train-logloss:0.251453
[20]	eval-logl

[66]	eval-logloss:0.931027	train-logloss:0.139051
[67]	eval-logloss:0.939112	train-logloss:0.136846
[68]	eval-logloss:0.959821	train-logloss:0.136114
[69]	eval-logloss:0.926966	train-logloss:0.13559
[70]	eval-logloss:0.926652	train-logloss:0.134102
[71]	eval-logloss:0.940041	train-logloss:0.132771
[72]	eval-logloss:0.933845	train-logloss:0.130916
[73]	eval-logloss:0.955431	train-logloss:0.129274
[74]	eval-logloss:0.995271	train-logloss:0.128101
[75]	eval-logloss:0.995243	train-logloss:0.127599
[76]	eval-logloss:0.989117	train-logloss:0.126951
[77]	eval-logloss:0.963273	train-logloss:0.126264
[78]	eval-logloss:0.954692	train-logloss:0.124915
[79]	eval-logloss:0.970215	train-logloss:0.123103
[80]	eval-logloss:0.98548	train-logloss:0.122465
[81]	eval-logloss:0.972448	train-logloss:0.12123
[82]	eval-logloss:0.982484	train-logloss:0.119883
[83]	eval-logloss:0.960613	train-logloss:0.118566
[84]	eval-logloss:0.931866	train-logloss:0.116879
[85]	eval-logloss:0.942242	train-logloss:0.115803
[86

[32]	eval-logloss:0.547177	train-logloss:0.214458
[33]	eval-logloss:0.55672	train-logloss:0.213217
[34]	eval-logloss:0.5637	train-logloss:0.210486
[35]	eval-logloss:0.558654	train-logloss:0.209065
[36]	eval-logloss:0.550586	train-logloss:0.207642
[37]	eval-logloss:0.55011	train-logloss:0.2068
[38]	eval-logloss:0.541546	train-logloss:0.204718
[39]	eval-logloss:0.546128	train-logloss:0.200268
[40]	eval-logloss:0.533367	train-logloss:0.198362
[41]	eval-logloss:0.526229	train-logloss:0.195419
[42]	eval-logloss:0.528913	train-logloss:0.192747
[43]	eval-logloss:0.554645	train-logloss:0.189837
[44]	eval-logloss:0.571807	train-logloss:0.18835
[45]	eval-logloss:0.58379	train-logloss:0.185895
[46]	eval-logloss:0.598113	train-logloss:0.183946
[47]	eval-logloss:0.598372	train-logloss:0.182615
[48]	eval-logloss:0.616728	train-logloss:0.180741
[49]	eval-logloss:0.600386	train-logloss:0.17765
[50]	eval-logloss:0.598327	train-logloss:0.175473
[51]	eval-logloss:0.599508	train-logloss:0.173996
[52]	eval

[97]	eval-logloss:1.01216	train-logloss:0.101905
[98]	eval-logloss:1.03386	train-logloss:0.100643
[99]	eval-logloss:1.03414	train-logloss:0.099483
[0]	eval-logloss:0.624216	train-logloss:0.560052
[1]	eval-logloss:0.574326	train-logloss:0.482692
[2]	eval-logloss:0.569936	train-logloss:0.435436
[3]	eval-logloss:0.576732	train-logloss:0.404673
[4]	eval-logloss:0.570418	train-logloss:0.38181
[5]	eval-logloss:0.55734	train-logloss:0.364591
[6]	eval-logloss:0.532165	train-logloss:0.34968
[7]	eval-logloss:0.550066	train-logloss:0.338956
[8]	eval-logloss:0.544298	train-logloss:0.327918
[9]	eval-logloss:0.553788	train-logloss:0.319914
[10]	eval-logloss:0.524396	train-logloss:0.311072
[11]	eval-logloss:0.53781	train-logloss:0.302468
[12]	eval-logloss:0.543571	train-logloss:0.297994
[13]	eval-logloss:0.536121	train-logloss:0.291206
[14]	eval-logloss:0.501521	train-logloss:0.285924
[15]	eval-logloss:0.506563	train-logloss:0.277507
[16]	eval-logloss:0.501836	train-logloss:0.27381
[17]	eval-logloss:

[63]	eval-logloss:0.928983	train-logloss:0.136743
[64]	eval-logloss:0.928806	train-logloss:0.135948
[65]	eval-logloss:0.921802	train-logloss:0.134608
[66]	eval-logloss:0.92892	train-logloss:0.133002
[67]	eval-logloss:0.900764	train-logloss:0.131675
[68]	eval-logloss:0.902398	train-logloss:0.130165
[69]	eval-logloss:0.90268	train-logloss:0.128849
[70]	eval-logloss:0.923366	train-logloss:0.12689
[71]	eval-logloss:0.926355	train-logloss:0.125247
[72]	eval-logloss:0.94374	train-logloss:0.12347
[73]	eval-logloss:0.935289	train-logloss:0.121775
[74]	eval-logloss:0.933607	train-logloss:0.120551
[75]	eval-logloss:0.932163	train-logloss:0.120126
[76]	eval-logloss:0.940961	train-logloss:0.118792
[77]	eval-logloss:0.939343	train-logloss:0.11785
[78]	eval-logloss:0.954454	train-logloss:0.116097
[79]	eval-logloss:0.958616	train-logloss:0.114326
[80]	eval-logloss:0.958692	train-logloss:0.114034
[81]	eval-logloss:0.953262	train-logloss:0.113329
[82]	eval-logloss:0.94957	train-logloss:0.111881
[83]	ev

...

[67]	eval-logloss:0.5989	train-logloss:0.127686
[68]	eval-logloss:0.596842	train-logloss:0.126103
[69]	eval-logloss:0.621164	train-logloss:0.125166
[70]	eval-logloss:0.620572	train-logloss:0.123716
[71]	eval-logloss:0.616786	train-logloss:0.123064
[72]	eval-logloss:0.616662	train-logloss:0.122004
[73]	eval-logloss:0.603212	train-logloss:0.119862
[74]	eval-logloss:0.580688	train-logloss:0.118844
[75]	eval-logloss:0.589873	train-logloss:0.117131
[76]	eval-logloss:0.595257	train-logloss:0.116432
[77]	eval-logloss:0.611036	train-logloss:0.114931
[78]	eval-logloss:0.607651	train-logloss:0.113802
[79]	eval-logloss:0.611766	train-logloss:0.112995
[80]	eval-logloss:0.63556	train-logloss:0.111384
[81]	eval-logloss:0.66547	train-logloss:0.109804
[82]	eval-logloss:0.676141	train-logloss:0.108355
[83]	eval-logloss:0.652512	train-logloss:0.106788
[84]	eval-logloss:0.66525	train-logloss:0.106453
[85]	eval-logloss:0.663692	train-logloss:0.104515
[86]	eval-logloss:0.663424	train-logloss:0.103069
[87]	

[33]	eval-logloss:0.654587	train-logloss:0.210343
[34]	eval-logloss:0.651935	train-logloss:0.208016
[35]	eval-logloss:0.638665	train-logloss:0.207177
[36]	eval-logloss:0.627491	train-logloss:0.203434
[37]	eval-logloss:0.626057	train-logloss:0.201339
[38]	eval-logloss:0.634005	train-logloss:0.196951
[39]	eval-logloss:0.633697	train-logloss:0.193064
[40]	eval-logloss:0.653578	train-logloss:0.188741
[41]	eval-logloss:0.657771	train-logloss:0.186596
[42]	eval-logloss:0.645823	train-logloss:0.183644
[43]	eval-logloss:0.643238	train-logloss:0.179903
[44]	eval-logloss:0.650896	train-logloss:0.176873
[45]	eval-logloss:0.63477	train-logloss:0.173972
[46]	eval-logloss:0.680323	train-logloss:0.172621
[47]	eval-logloss:0.683095	train-logloss:0.169628
[48]	eval-logloss:0.663183	train-logloss:0.167263
[49]	eval-logloss:0.651159	train-logloss:0.166575
[50]	eval-logloss:0.653315	train-logloss:0.163858
[51]	eval-logloss:0.652547	train-logloss:0.161558
[52]	eval-logloss:0.662279	train-logloss:0.159666
[

[98]	eval-logloss:0.898085	train-logloss:0.097233
[99]	eval-logloss:0.896891	train-logloss:0.097
[0]	eval-logloss:0.626222	train-logloss:0.553127
[1]	eval-logloss:0.628065	train-logloss:0.475199
[2]	eval-logloss:0.583909	train-logloss:0.426981
[3]	eval-logloss:0.574262	train-logloss:0.394147
[4]	eval-logloss:0.603316	train-logloss:0.36878
[5]	eval-logloss:0.641112	train-logloss:0.350022
[6]	eval-logloss:0.603588	train-logloss:0.333127
[7]	eval-logloss:0.627926	train-logloss:0.319865
[8]	eval-logloss:0.643646	train-logloss:0.309353
[9]	eval-logloss:0.674797	train-logloss:0.30052
[10]	eval-logloss:0.648098	train-logloss:0.289412
[11]	eval-logloss:0.66352	train-logloss:0.281632
[12]	eval-logloss:0.639182	train-logloss:0.275587
[13]	eval-logloss:0.649217	train-logloss:0.271518
[14]	eval-logloss:0.671356	train-logloss:0.265461
[15]	eval-logloss:0.686557	train-logloss:0.258352
[16]	eval-logloss:0.672808	train-logloss:0.25432
[17]	eval-logloss:0.657275	train-logloss:0.251576
[18]	eval-logloss

[64]	eval-logloss:0.485406	train-logloss:0.139205
[65]	eval-logloss:0.485354	train-logloss:0.138425
[66]	eval-logloss:0.504948	train-logloss:0.136148
[67]	eval-logloss:0.503075	train-logloss:0.134128
[68]	eval-logloss:0.49153	train-logloss:0.132954
[69]	eval-logloss:0.490497	train-logloss:0.132386
[70]	eval-logloss:0.483308	train-logloss:0.131215
[71]	eval-logloss:0.486145	train-logloss:0.129461
[72]	eval-logloss:0.48662	train-logloss:0.128491
[73]	eval-logloss:0.518713	train-logloss:0.127316
[74]	eval-logloss:0.516767	train-logloss:0.125799
[75]	eval-logloss:0.521623	train-logloss:0.124408
[76]	eval-logloss:0.513934	train-logloss:0.12294
[77]	eval-logloss:0.51335	train-logloss:0.122494
[78]	eval-logloss:0.51348	train-logloss:0.121732
[79]	eval-logloss:0.517883	train-logloss:0.119421
[80]	eval-logloss:0.514943	train-logloss:0.118837
[81]	eval-logloss:0.500431	train-logloss:0.117083
[82]	eval-logloss:0.502462	train-logloss:0.116455
[83]	eval-logloss:0.510221	train-logloss:0.114574
[84]	

[29]	eval-logloss:0.448336	train-logloss:0.214429
[30]	eval-logloss:0.44773	train-logloss:0.212781
[31]	eval-logloss:0.462317	train-logloss:0.208523
[32]	eval-logloss:0.445354	train-logloss:0.206761
[33]	eval-logloss:0.467084	train-logloss:0.204999
[34]	eval-logloss:0.458317	train-logloss:0.203866
[35]	eval-logloss:0.465354	train-logloss:0.201247
[36]	eval-logloss:0.465088	train-logloss:0.200483
[37]	eval-logloss:0.449798	train-logloss:0.196934
[38]	eval-logloss:0.442234	train-logloss:0.194422
[39]	eval-logloss:0.444376	train-logloss:0.193667
[40]	eval-logloss:0.441075	train-logloss:0.191509
[41]	eval-logloss:0.440913	train-logloss:0.189991
[42]	eval-logloss:0.431157	train-logloss:0.189424
[43]	eval-logloss:0.437368	train-logloss:0.187768
[44]	eval-logloss:0.433595	train-logloss:0.184877
[45]	eval-logloss:0.429579	train-logloss:0.183541
[46]	eval-logloss:0.432377	train-logloss:0.18061
[47]	eval-logloss:0.435062	train-logloss:0.178257
[48]	eval-logloss:0.421528	train-logloss:0.174494
[4

In [17]:
assert sum(min_scr <= midpt) >= (0.7 * len(sample_indices)), "Incorrect range (mean - 3*std) to (mean + 3*std)"

In [18]:
assert sum(max_scr >= midpt) >= (0.7 * len(sample_indices)), "Incorrect range (mean - 3*std) to (mean + 3*std)"

In [19]:
assert sum(length < 2*avg_length) >= (0.7 * len(sample_indices)), "Incorrect length of range (mean - 3*std) to (mean + 3*std)"

In [20]:
# Hidden Tests Here
#
# AUTOGRADER TEST - DO NOT REMOVE
#


In [21]:
# Hidden Tests Here
#
# AUTOGRADER TEST - DO NOT REMOVE
#


In [22]:
# Hidden Tests Here
#
# AUTOGRADER TEST - DO NOT REMOVE
#


In [23]:
# Hidden Tests Here
#
# AUTOGRADER TEST - DO NOT REMOVE
#


In [24]:
# Hidden Tests here
#
# AUTOGRADER TEST - DO NOT REMOVE
#


In [25]:
# Hidden Tests here
#
# AUTOGRADER TEST - DO NOT REMOVE
#


In [26]:
# Hidden Tests here
#
# AUTOGRADER TEST - DO NOT REMOVE
#


In [27]:
# Hidden Tests here
#
# AUTOGRADER TEST - DO NOT REMOVE
#
